In [0]:
%run "/Workspace/Users/silvester.g@diggibyte.com/DataBricks_Assignment/src/question1/source_to_bronze/utils"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Define schemas (custom)
employee_schema = StructType([
    StructField("EmployeeID", IntegerType(), True),
    StructField("EmployeeName", StringType(), True),
    StructField("DepartmentID", StringType(), True),
    StructField("CountryID", StringType(), True),
    StructField("Salary", DoubleType(), True),
    StructField("Age", IntegerType(), True)
])

department_schema = StructType([
    StructField("DepartmentID", IntegerType(), True),
    StructField("DepartmentName", StringType(), True)
])

country_schema = StructType([
    StructField("CountryID", IntegerType(), True),
    StructField("CountryName", StringType(), True)
])

# Read bronze CSVs
employee_df = spark.read.option("header", True).schema(employee_schema).csv("/Volumes/assignment/default/write/employee")
department_df = spark.read.option("header", True).schema(department_schema).csv("/Volumes/assignment/default/write/department")
country_df = spark.read.option("header", True).schema(country_schema).csv("/Volumes/assignment/default/write/country")

# Convert to snake_case and add load_date
employee_df = rename_columns_to_snake_case(employee_df)
employee_df = add_load_date(employee_df)

# Create a database (if you want)
spark.sql("CREATE DATABASE IF NOT EXISTS assignment.Employee_info")

# Write as Delta table inside that database
employee_df.write.format("delta").mode("overwrite").saveAsTable("assignment.Employee_info.dim_employee")


print("✅ Data successfully written to silver layer (Delta)")


In [0]:
%sql
SELECT * 
FROM assignment.employee_info.dim_employee